In [5]:
import qlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
import os
from qlib.data import D
from qlib.config import REG_CN
from qlib.contrib.data.handler import Alpha158
from qlib.data.filter import NameDFilter, ExpressionDFilter
import warnings
warnings.filterwarnings('ignore')

In [2]:
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.utils import init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord

ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).
ModuleNotFoundError.  PyTorch models are skipped (optional: maybe installing pytorch can fix it).


In [3]:
qlib.init(provider_uri='C:/Users/tantra/Desktop/data/day1_data_qlib', region=REG_CN)


[15776:MainThread](2025-04-28 09:37:31,240) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[15776:MainThread](2025-04-28 09:37:31,244) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[15776:MainThread](2025-04-28 09:37:31,248) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/tantra/Desktop/data/day1_data_qlib')}


In [7]:
expressionDFilter = ExpressionDFilter(rule_expression='$volume!=0')
instruments = D.instruments(market = 'all', filter_pipe=[expressionDFilter])

In [8]:
data_close = D.features(instruments = instruments, fields = ['$close'], start_time = '2022-01-01', end_time = '2025-03-31', freq = '1day').unstack().T.droplevel(0)
data_close

instrument,110038.SH,110043.SH,110044.SH,110045.SH,110047.SH,110048.SH,110052.SH,110053.SH,110055.SH,110056.SH,...,128136.SZ,128137.SZ,128138.SZ,128139.SZ,128140.SZ,128141.SZ,128142.SZ,128143.SZ,128144.SZ,128145.SZ
datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-04,150.570007,122.300003,206.880005,118.949997,122.070000,246.820007,118.250000,120.550003,204.940002,135.229996,...,127.000000,156.000000,116.560997,118.000000,164.000000,130.300003,123.697998,129.990005,142.009995,143.660995
2022-01-05,144.690002,123.970001,208.020004,119.779999,122.169998,236.690002,119.540001,121.580002,205.139999,134.250000,...,125.000000,154.520996,116.809998,117.885002,161.210007,144.009995,124.699997,128.880005,140.110992,140.139999
2022-01-06,147.009995,123.940002,205.300003,119.809998,122.519997,225.630005,114.129997,121.440002,204.080002,135.839996,...,124.130997,151.800003,118.212997,117.989998,161.000000,145.367004,126.080002,130.899994,142.169998,142.259995
2022-01-07,143.330002,124.480003,206.070007,119.010002,123.940002,222.000000,114.800003,122.019997,197.070007,135.800003,...,125.000000,150.042999,118.323997,119.000000,159.600006,143.604004,124.970001,130.298996,145.789993,140.190002
2022-01-10,142.199997,125.589996,204.270004,119.010002,123.870003,217.990005,114.800003,123.489998,196.100006,136.119995,...,125.489998,150.100006,118.870003,126.000000,158.880005,145.699997,125.000000,130.250000,145.927002,140.850006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120.252998,117.011002,125.448997,NaN,NaN,130.003998,120.769997,NaN,131.263000,NaN
2025-03-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120.571999,117.300003,125.705002,NaN,NaN,130.029999,122.276001,NaN,130.210007,NaN
2025-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120.442001,117.139000,125.890999,NaN,NaN,130.675003,122.278999,NaN,135.500000,NaN


In [9]:
ret5 = (data_close / data_close.shift(5) -1).dropna(how = 'all')
ret5

instrument,110038.SH,110043.SH,110044.SH,110045.SH,110047.SH,110048.SH,110052.SH,110053.SH,110055.SH,110056.SH,...,128136.SZ,128137.SZ,128138.SZ,128139.SZ,128140.SZ,128141.SZ,128142.SZ,128143.SZ,128144.SZ,128145.SZ
datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-11,-0.054858,0.027473,-0.006622,0.005801,0.004833,-0.134065,-0.010909,0.031107,-0.060896,0.005694,...,-0.020197,-0.059359,0.019209,0.069068,-0.051829,0.108289,0.010445,0.007000,0.046243,-0.027572
2022-01-12,-0.019421,0.017101,-0.012643,-0.002254,0.008349,-0.062487,-0.012213,0.029363,-0.051185,0.011620,...,0.013592,0.004873,0.018389,0.077067,-0.026115,-0.007361,-0.001604,0.022657,0.063086,0.030184
2022-01-13,-0.053602,0.011538,-0.002728,-0.004924,0.004407,-0.048619,0.013932,0.029068,-0.055566,-0.007730,...,0.006719,-0.001976,0.004898,0.076091,-0.055901,-0.025879,-0.012532,-0.001765,0.031828,0.020666
2022-01-14,-0.029722,-0.003454,-0.024846,-0.013528,-0.026545,-0.043198,-0.006620,0.002787,-0.035419,-0.003535,...,-0.000320,0.013709,-0.011190,0.067059,-0.058841,-0.014651,-0.014804,0.010737,-0.019137,0.021471
2022-01-17,-0.021660,-0.015766,-0.013756,-0.013444,-0.028498,-0.025414,0.017683,-0.012714,-0.023355,0.013371,...,-0.001195,0.058627,-0.010768,0.025230,-0.002392,0.008236,-0.008008,0.019570,-0.020058,0.020092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.018311,-0.031927,-0.007900,NaN,NaN,-0.062967,-0.027374,NaN,-0.015946,NaN
2025-03-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.005575,-0.020868,0.000048,NaN,NaN,-0.044950,-0.006492,NaN,-0.006637,NaN
2025-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001347,-0.018032,0.002317,NaN,NaN,-0.008054,-0.005498,NaN,0.047108,NaN


In [ ]:
# 先将DataFrame保存为CSV（需符合QLib格式）
ret5.to_csv("C:/Users/tantra/Desktop/ret5_temp.csv")

# 转换为.bin格式（注意路径替换为您的实际目录）
python scripts/dump_bin.py dump_all \
    --csv_path C:/Users/tantra/Desktop/ret5_temp.csv \
    --qlib_dir "C:/Users/tantra/Desktop/data/因子bin" \
    --include_fields ret5 \
    --freq 1day

SyntaxError: invalid syntax (4083959133.py, line 5)

In [36]:
for csvname in os.listdir('C:/Users/tantra/Desktop/day1_data_raw'):
    stock = csvname[:9]
    df = pd.read_csv(f'C:/Users/tantra/Desktop/day1_data_raw/{csvname}', parse_dates=['date'])
    df.set_index('date', inplace = True)
    try:
        df['ret5'] = ret5.loc[:,stock]
    except:
        df['ret5'] = np.nan
    df.to_csv(f'C:/Users/tantra/Desktop/因子csv保存尝试/{csvname}')
        

In [19]:
df = pd.read_csv(f'C:/Users/tantra/Desktop/day1_data_raw/110038.SH.csv', parse_dates=['date'])
df.set_index('date',inplace = True)
df['ret5'] = ret5.loc[:,'110038.SH']

In [ ]:
df.to_csv('C:/Users/tantra/Desktop/因子保存110038.SH.csv')

In [31]:
import qlib
from qlib.data import D
from qlib.data.dataset.handler import DataHandlerLP
from qlib.data.data import Cal
from qlib.data.filter import NameDFilter
import pandas as pd
import os
import subprocess
import requests
import json
import time

In [40]:
'''
# 将原始数据转换成中间数据（csv）
'''
# for csvname in os.listdir('C:/Users/tantra/Desktop/因子csv保存尝试'):
#     instrument = csvname[:9]
#     print(f"正在转换中间文件：{instrument}...")
#     df_tmp = pd.read_csv(f'C:/Users/tantra/Desktop/因子csv保存尝试/{instrument}.csv')
#     df_tmp['datetime'] = pd.to_datetime(df_tmp['date'], format='%Y-%m-%d')
#     df_tmp['instrument'] = instrument
#     df_tmp = df_tmp[['datetime', 'instrument', 'open', 'high', 'low', 'close', 'volume', 'ret5']]
#     df_tmp.set_index(['datetime', 'instrument'], inplace=True)
#     df_tmp.to_csv(f'C:/Users/tantra/Desktop/因子csv保存qlib/{instrument}.csv')


'''
csv转bin
其中
E:/PY_qlib_env/.venv/Scripts/python.exe 指python路径
E:/PY_qlib_env/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all 指qlib的dump_bin.py路径
--csv_path 原始数据路径
--qlib_dir 转换后数据路径
--freq 1min 转换后数据频率
--date_field_name datetime 转换后数据时间字段
--symbol_field_name instrument 转换后数据股票代码字段
--include_fields open,high,low,close,volume 转换后数据包含字段
'''
print(f"正在执行把csv文件转换为bin文件...")
subprocess.run(f"D:/anaconda/envs/py10/python.exe C:/Users/tantra/Desktop/data/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all --csv_path C:/Users/tantra/Desktop/因子csv保存qlib --qlib_dir C:/Users/tantra/Desktop/因子qlib --freq 1day --date_field_name datetime --symbol_field_name instrument --include_fields open,high,low,close,volume,ret5")

正在执行把csv文件转换为bin文件...


CompletedProcess(args='D:/anaconda/envs/py10/python.exe C:/Users/tantra/Desktop/data/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all --csv_path C:/Users/tantra/Desktop/因子csv保存qlib --qlib_dir C:/Users/tantra/Desktop/因子qlib --freq 1day --date_field_name datetime --symbol_field_name instrument --include_fields open,high,low,close,volume,ret5', returncode=0)